# Making a trial by trial fit of faceWord dataand using that for searchligth classificiation

This tutorial will load models and data from tutorial 13 and change the design matrix to one that gives a beta estimate for each trial.

In [5]:
# import some functionality
from datetime import datetime

now = datetime.now()
print('Starting cell:',now.strftime("%H:%M:%S"))
import os
import pip
os.system('python -m pip install numpy')
os.system('python -m pip install matplotlib')
os.system('python -m pip install scipy')
os.system('python -m pip install panda')
os.system('python -m pip install nilearn')
os.system('python -m pip install sklearn')
os.system('python -m pip install pickle')
os.system('python -m pip install atlasreader')
os.system('python -m pip install seaborn')

#import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from scipy.ndimage import gaussian_filter

%matplotlib inline

now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

Starting cell: 11:18:25


/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Finishing cell: 11:18:44


In [6]:
import pickle

now = datetime.now()
print('Starting cell:',now.strftime("%H:%M:%S"))

# Getting the first level models from tutorial 13:
f = open('/work/82777/WordFace_first_level_models.pkl', 'rb')
models, models_run_imgs, models_events, models_confounds = pickle.load(f)
f.close()

now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

Starting cell: 11:19:15
Finishing cell: 11:20:17


## Saving/retrieving the fitted models and design matrices

In [2]:
import pickle

# Getting back the objects:
f = open('/work/82777/WordFace_first_level_models_all_trials_all_par.pkl', 'rb')
models_trials_all, lsa_dm = pickle.load(f)
f.close()

#print(model1[0])
#now = datetime.now()
#print('Finishing cell:',now.strftime("%H:%M:%S"))

/opt/conda/lib/python3.10/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


## Load z maps and classification labels for each participant


In [7]:
# Getting back the objects:
N_par=len(models_events) # Number of participants
z_maps_all= np.empty((N_par, 0)).tolist()
conditions_label_all= np.empty((N_par, 0)).tolist()
for i in rante(0,5)#range(N_par):
    text = "Loading file %d\n" % (i+1)
    print(text)
    file_name='/work/82777/WordFace_first_level_z_maps_all_trials_all_par_'+str(i)+'.pkl'
    f = open(file_name, 'rb')
    conditions_label_temp, z_maps_temp = pickle.load(f)
    conditions_label_all[i]=conditions_label_temp
    z_maps_all[i]=z_maps_temp
    f.close()

print(conditions_label_all[9])
            
now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

Loading file 1

Loading file 2

Loading file 3

Loading file 4

Loading file 5

Loading file 6

Loading file 7

Loading file 8

Loading file 9

Loading file 10

Loading file 11

Loading file 12

Loading file 13

Loading file 14

Loading file 15

Loading file 16

Loading file 17

Loading file 18

Loading file 19

Loading file 20

Loading file 21

Loading file 22

['image_pos', 'image_neg', 'image_pos', 'image_pos', 'image_neg', 'image_pos', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_neg', 'image_neg', 'image_neg', 'image_neg', 'image_pos', 'image_pos', 'image_neg', 'image_pos', 'image_pos', 'image_neg', 'image_neg', 'image_neg', 'image_pos', 'image_neg', 'image_neg', 'image_neg', 'image_pos', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_neg', 'image_neg', 'image_pos', 'image_pos', 'image_pos', 'image_pos', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_pos', 'image_neg', 'image_pos', 'image_pos

## Reshape data for classification

In [8]:

now = datetime.now()
print('Starting cell:',now.strftime("%H:%M:%S"))

import numpy as np
#from nilearn import datasets
from nilearn.image import new_img_like, load_img, index_img, clean_img
from sklearn.model_selection import train_test_split, GroupKFold

N_par=len(models_events) # Number of participants
z_maps_img_all= np.empty((N_par, 0)).tolist()
z_maps_conc= np.empty((N_par, 0)).tolist()
conditions= np.empty((N_par, 0)).tolist()

for i in range(0,5):#range(N_par):
    n_trials=len(conditions_label_all[i])
    text = "Reshaping participant %d\n" % (i+1)
    print(text)


    # Reshaping data------------------------------
    from nilearn.image import index_img, concat_imgs
    idx_neg=[int(iii) for iii in range(len(conditions_label_all[i])) if conditions_label_all[i][iii]=='image_neg']
    idx_pos=[int(iii) for iii in range(len(conditions_label_all[i])) if conditions_label_all[i][iii]=='image_pos']

    #Concatenate trials
    idx=np.concatenate((idx_neg, idx_pos))

    conditions[i]=np.array(conditions_label_all[i])[idx]
    z_maps_conc[i]=concat_imgs(z_maps_all[i])
    print(z_maps_conc[i].shape)
    z_maps_img_all[i] = index_img(z_maps_conc[i], idx)

    print(z_maps_img_all[i].shape)

now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

Starting cell: 11:29:35
Reshaping participant 1

(78, 92, 65, 720)
(78, 92, 65, 360)
Finishing cell: 11:29:37


## create training and testing vars on the basis of class labels

In [9]:
now = datetime.now()
print('Starting cell:',now.strftime("%H:%M:%S"))

N_par=len(models_events) # Number of participants
idx1= np.empty((N_par, 0)).tolist()
idx2= np.empty((N_par, 0)).tolist()
conditions1= np.empty((N_par, 0)).tolist()
conditions2= np.empty((N_par, 0)).tolist()
fmri_img1= np.empty((N_par, 0)).tolist()
fmri_img2= np.empty((N_par, 0)).tolist()

for i in range (0,5):#(N_par):
    text = "Creating trian/test data for participant %d\n" % (i+1)
    print(text)
    idx2[i]=np.arange(conditions[i].shape[0])
    # create training and testing vars on the basis of class labels
    idx1[i],idx2[i], conditions1[i],  conditions2[i] = train_test_split(idx2[i],conditions[i], test_size=0.2)
    
    # Reshaping data------------------------------
    from nilearn.image import index_img
    fmri_img1[i] = index_img(z_maps_img_all[i], idx1[i])
    fmri_img2[i] = index_img(z_maps_img_all[i], idx2[i])

now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

Starting cell: 11:30:20
Creating trian/test data for participant 1

Finishing cell: 11:30:21


In [16]:
print(len(conditions[0]))
print(len(idx1[0]))
print(len(idx2[0]))

360
288
72


## Make a classification on the 2nd split using the best voxels

In [ ]:
import nilearn
from nilearn import plotting
atlas_name = "cort-maxprob-thr50-2mm"

atlas = nilearn.datasets.fetch_atlas_harvard_oxford(atlas_name, data_dir="/work/fMRI Assignment/neuro_exam/atlases", symmetric_split=False, resume=True, verbose=1)
atlas2 = nilearn.datasets.fetch_atlas_harvard_oxford(atlas_name, data_dir="/work/fMRI Assignment/neuro_exam/atlases", symmetric_split=False, resume=True, verbose=1)
# PREPARING ATLAS
atlas_img = atlas["maps"]
# Get labels
labels = atlas.labels
atlas_data = atlas_img.get_fdata()

from nilearn.image import resample_img as resample

target_affine = test_img.affine

shape = test_img.shape
target_shape = [shape[0], shape[1], shape[2]]

resampled_atlas =resample(atlas_img,target_affine = target_affine, target_shape = target_shape, interpolation = 'nearest')

resamp_data = resampled_atlas.get_fdata()

mask = resamp_data.copy()


x = resamp_data.shape[0]
y = resamp_data.shape[1]
z = resamp_data.shape[2]

# This will set all elements of atlas_data to either 0 or 1.
m = 0
for i in range(0, x):
    for j in range(0, y):
        for k in range(0, z):
            if resamp_data[i][j][k] in [5,6,40]:
                m += 1
                mask[i][j][k] = 1
            else:
                mask[i][j][k] = 0
print(m)

mask_nifti = nib.Nifti1Image(mask, affine = target_affine)
plotting.plot_roi(mask_nifti)
plotting.plot_roi(atlas_img)

### Perform a permutation test

In [ ]:
now = datetime.now()
print('Starting cell:',now.strftime("%H:%M:%S"))

from sklearn.model_selection import permutation_test_score
from sklearn import LinearSVC

lin_clf = LinearSVC(penalty = "l2")

for i in range(0,5):#(N_par):
    
    masker = NiftiMasker(mask_img=process_mask2_img[i], standardize=False)
    # We use masker to retrieve a 2D array ready
    # for machine learning with scikit-learn
    fmri_masked_train = masker.fit_transform(fmri_img1[i])
    lin_clf.fit(fmri_masked_train)


for i in range(0,5):#(N_par):
    score[i], permutation_scores[i], pvalue[i]= permutation_test_score(
        lin_clf, fmri_masked[i], conditions2[i], cv=10, n_permutations=1000, 
        n_jobs=-1, random_state=0, verbose=0, scoring=None)
    print("Classification Accuracy for participant %d: %s (pvalue : %s)" % (i,score, pvalue))
 
# Saving the objects:
f = open('/work/MikkelWallentin#6287/WordFace_first_level_models_all_trials_classified.pkl', 'wb')
pickle.dump([searchlight_all, searchlight_img, cut, perc, process_mask2_img, score,permutation_score,pvalue], f)
f.close()


now = datetime.now()
print('Finishing cell:',now.strftime("%H:%M:%S"))

### View a histogram of permutation scores

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#How many classes
#n_classes = np.unique(conditions2).size

#plt.hist(permutation_scores, 20, label='Permutation scores',
#         edgecolor='black')
#ylim = plt.ylim()
#plt.plot(2 * [score], ylim, '--g', linewidth=3,
#         label='Classification Score'
#         ' (pvalue %s)' % pvalue)
#plt.plot(2 * [1. / n_classes], ylim, '--k', linewidth=3, label='Chance level')

#plt.ylim(ylim)
#plt.legend()
#plt.xlabel('Score')
#plt.show()